# SEIRX model runs for test technology and screening frequency investigation

In [29]:
import networkx as nx
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
import os
from os.path import expanduser
home = expanduser("~")
os.chdir(home+'/agent_based_COVID_SEIRX/src/')
os.getcwd()
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
from scseirx import analysis_functions as af # custom analysis functions
import data_creation_functions as dcf

from importlib import reload

## Parameters

### Simulation parameters and measures in place

In [30]:

os.getcwd()

'/Users/ledebur/agent_based_COVID_SEIRX/src'

In [18]:
with open(home+'/nursing_home_SEIRX/params/screening_frequency_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open(home+'/nursing_home_SEIRX/params/screening_frequency_measures.json') as json_file:
    measures = json.load(json_file)


### Screening parameters

In [19]:
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen', 'same_day_LAMP', 'same_day_PCR',\
              'one_day_PCR', 'two_day_PCR']

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

# resident and employee streening intervals (in days)
e_screen_range = [2, 3, 7, None]
r_screen_range = [2, 3, 7, None]

screening_params = [(i, j, k, l)\
              for i in test_types \
              for j in index_cases \
              for k in e_screen_range \
              for l in r_screen_range]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 160 parameter combinations


In [33]:
def run(param_list, N_runs=10000):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''

    # paths for data I/O
    contact_network_src = home+'/nursing_home_SEIRX/data/contact_networks'

    # extract the simulation parameters from the parameter list
    test_type, index_case, e_screen_range, r_screen_range = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, test_type, index_case, 
                e_screen_range, r_screen_range, measures,
                simulation_params, contact_network_src)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, test_type, index_case, 
                                e_screen_range, r_screen_range)

    return row

In [31]:
os.chdir()

In [ ]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '..data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case',
              'resident_screen_interval', 'employee_screen_interval']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_test_technology_screening_interval.csv'),
               index=False)
results.head(3)

  8%|▊         | 12/160 [15:00<1:23:30, 33.85s/it] 